# VAE Training - Faces dataset

## imports

In [1]:
import os, time
from glob import glob
import numpy as np

from model.VariationalAutoencoder import VariationalAutoencoder
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


# environment variables

In [2]:
cloud_training = False

if (cloud_training) :
    DATA_FOLDER = '/data'
    RUN_FOLDER = '/tmp/'
else:
    DATA_FOLDER = '/Volumes/My Passport/PFM/fastcheck'
    RUN_FOLDER = '/Volumes/My Passport/PFM/run/'

DATETIME = time.strftime("%Y%m%d-%H%M%S")

# VAE

## model variables

In [3]:
LEARNING_RATE = 0.0005
R_LOSS_FACTOR = 1000

run_id = '0001'
data_name = 'cells'
RUN_FOLDER += '_'.join([run_id, data_name, DATETIME])

INPUT_DIM = (128,128,3)

if (cloud_training) :
    BATCH_SIZE = 200
    EPOCHS = 50
else:
    BATCH_SIZE = 5
    EPOCHS = 5

## training setup

In [4]:
t = VariationalAutoencoder()

t.compile(learning_rate=LEARNING_RATE, r_loss_factor=R_LOSS_FACTOR)

t.encoder.summary()
t.decoder.summary()

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))
    

filenames = np.array(glob(os.path.join(DATA_FOLDER, '*/*.png')))

NUM_IMAGES = len(filenames)

print('Total images: ' + str(NUM_IMAGES))

steps_per_epoch = NUM_IMAGES / BATCH_SIZE
data_gen = ImageDataGenerator(rescale=1./255) # 8 BITS X CHANNEL
data_flow = data_gen.flow_from_directory(DATA_FOLDER
                                         , target_size = INPUT_DIM[:2]
                                         , batch_size = BATCH_SIZE
                                         , shuffle = True
                                         , class_mode = 'input'
                                         , subset = "training")

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
encoder_conv_1 (Conv2D)         (None, 64, 64, 32)   896         encoder_input[0][0]              
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 64, 64, 32)   0           encoder_conv_1[0][0]             
__________________________________________________________________________________________________
encoder_conv_2 (Conv2D)         (None, 32, 32, 64)   18496       leaky_re_lu_1[0][0]              
____________________________________________________________________________________________

## train

In [5]:
t.train_with_generator(data_flow = data_flow, epochs = EPOCHS, steps_per_epoch = NUM_IMAGES / BATCH_SIZE, run_folder = RUN_FOLDER, print_every_n_batches = 10)

Epoch 1/5
5/4 [===================================] - 29s 6s/step - loss: 8.5138 - vae_r_loss: 8.6904 - vae_kl_loss: 0.0541

Epoch 00001: saving model to /Volumes/My Passport/PFM/run/0001_cells_20200408-131245/weights/weights-001-8.74.h5

Epoch 00001: saving model to /Volumes/My Passport/PFM/run/0001_cells_20200408-131245/weights/weights.h5
Epoch 2/5
4/4 [===========================>..] - ETA: 0s - loss: 5.0816 - vae_r_loss: 5.0663 - vae_kl_loss: 0.0153

KeyboardInterrupt: 